In [8]:
import requests
from typing import Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain.utilities import DuckDuckGoSearchAPIWrapper, WikipediaAPIWrapper
from bs4 import BeautifulSoup
from dotenv import load_dotenv

load_dotenv()


llm = ChatOpenAI(temperature=0.1, model_name="gpt-3.5-turbo-1106")


class WikipediaSearchArgsSchema(BaseModel):
    query: str = Field(description="Search term to look up on Wikipedia.")

class WikipediaSearchTool(BaseTool):
    name = "WikipediaSearchTool"
    description = "Searches Wikipedia for a given query and returns summary."
    args_schema: Type[WikipediaSearchArgsSchema] = WikipediaSearchArgsSchema

    def _run(self, query):
        wiki = WikipediaAPIWrapper()
        return wiki.run(query)


class DuckDuckGoSearchArgsSchema(BaseModel):
    query: str = Field(description="Search term to look up on DuckDuckGo.")

class DuckDuckGoSearchTool(BaseTool):
    name = "DuckDuckGoSearchTool"
    description = "Performs a DuckDuckGo web search and returns relevant results."
    args_schema: Type[DuckDuckGoSearchArgsSchema] = DuckDuckGoSearchArgsSchema

    def _run(self, query: str):
        ddg = DuckDuckGoSearchAPIWrapper()
        return ddg.run(query)


class WebScrapeArgsSchema(BaseModel):
    url: str = Field(description="The URL of the website to scrape.")

class WebScrapeTool(BaseTool):
    name = "WebScrapeTool"
    description = "Scrapes text content from the specified URL."
    args_schema: Type[WebScrapeArgsSchema] = WebScrapeArgsSchema

    def _run(self, url: str):
        response = requests.get(url)
        soup = BeautifulSoup(response.text, "html.parser")
        return soup.get_text()


research_agent = initialize_agent(
    tools=[
        WikipediaSearchTool(),
        DuckDuckGoSearchTool(),
        WebScrapeTool(),
    ],
    llm=llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True,
    handle_parsing_errors=True,
)


response = research_agent.invoke("Research about the XZ backdoor")
with open("assignment9_result.txt", "w") as f:
    f.write(response["output"])



> Entering new AgentExecutor chain...

Invoking: `WikipediaSearchTool` with `{'query': 'XZ backdoor'}`


Page: XZ Utils backdoor
Summary: In February 2024, a malicious backdoor was introduced to the Linux build of the xz utility within the liblzma library in versions 5.6.0 and 5.6.1 by an account using the name "Jia Tan". The backdoor gives an attacker who possesses a specific Ed448 private key remote code execution capabilities on the affected Linux system. The issue has been given the Common Vulnerabilities and Exposures number CVE-2024-3094 and has been assigned a CVSS score of 10.0, the highest possible score.
While xz is commonly present in most Linux distributions, at the time of discovery the backdoored version had not yet been widely deployed to production systems, but was present in development versions of major distributions. The backdoor was discovered by the software developer Andres Freund, who announced his findings on 29 March 2024.



Page: XZ Utils
Summary: XZ Utils 